In [1]:
import pandas as pd

# 데이터 확인하기

In [2]:
tokens = pd.read_csv('csv_data/tokens.csv', encoding='utf-8-sig')

In [4]:
sentences = pd.read_csv('csv_data/dialect_standard_pairs.csv', encoding='utf-8-sig', engine='python')

In [5]:
sentences.head()

,region,dialect_form,standard_form
0,강원,난 니가 내 채고 친한 친구였으문 좋겠단 말이야.,난 네가 내 가장 친한 친구였으면 좋겠단 말이야.
1,강원,내 마옴을 받아달라는 거 아니야.,내 마음을 받아달라는 거 아니야.
2,강원,내 마옴을 니가 알아달라는 것 뿐이야.,내 마음을 네가 알아달라는 것 뿐이야.
3,강원,내가 니 옆에 있다는 것을 기억해 주면 돼.,내가 네 옆에 있다는 것을 기억해 주면 돼.
4,강원,내가 이딴 고백을 했다구 날 멀리 하진 마.,내가 이런 고백을 했다구 날 멀리 하진 마.


In [6]:
tokens.head()

,Region,Eojeol,Standard,IsDialect
0,강원,난,난,0
1,강원,니가,네가,1
2,강원,내,내,0
3,강원,채고,가장,1
4,강원,친한,친한,0


## 데이터 전처리

### 1. 결측치 확인

- 지역 데이터 양 확인

In [7]:
sentences['region'].value_counts()

region
제주                            3096648
경상                            2088717
강원                            1850036
전라                            1654555
전남                             582767
충청                             328548
 전                              17702
솔트                              11136
 제                              10589
38                                822
 솔                                478
원시                                385
지역                                215
 아니면 가지 말까요?                        1
것이 비스무리하네.                          1
것이 비슷하다.                            1
 간물을 먹었다오.                          1
 바닷물을 먹었다오.                         1
 늦게 인나쿠니 벼 알기이에 하마 발부리가났어.          1
 늦게 일어나니 벼 알갱이에 벌써 싹이났어.            1
 아니면 가지 말라와?                        1
Name: count, dtype: int64

In [8]:
tokens['Region'].value_counts()

Region
경상    16915800
제주    16371447
전라    12227348
강원    10165285
전남     4620291
충청     2990109
 전      105375
솔트       77495
 제       57360
38        4168
원시        3680
 솔        2745
지역        1604
제            1
Name: count, dtype: int64

- 전남, 전은 전라로, 제는 제주로 통합

In [9]:
tokens_revised = tokens.replace({'Region': {'전남': '전라', ' 전': '전라', ' 제': '제주'}})

In [10]:
is_province = tokens_revised['Region'].str.contains('제주|전라|경상|충청|강원')

tokens_revised = tokens_revised[is_province]

In [11]:
tokens_revised['Region'].value_counts()

Region
전라    16953014
경상    16915800
제주    16428807
강원    10165285
충청     2990109
Name: count, dtype: int64

- sentence도 동일 작업 실시

In [12]:
sentences_revised = sentences.replace({'region': {'전남': '전라', ' 전': '전라', ' 제': '제주'}})

In [13]:
is_province_sentence = sentences_revised['region'].str.contains('제주|전라|경상|강원|충청')

In [14]:
sentences_revised = sentences_revised[is_province_sentence]

In [15]:
sentences_revised['region'].value_counts()

region
제주    3107237
전라    2255024
경상    2088717
강원    1850036
충청     328548
Name: count, dtype: int64

### 2. 원핫인코딩

In [16]:
token_region_dummies = pd.get_dummies(tokens_revised, columns=['Region'])

token_region_dummies.head()

,Eojeol,Standard,IsDialect,Region_강원,Region_경상,Region_전라,Region_제주,Region_충청
0,난,난,0,True,False,False,False,False
1,니가,네가,1,True,False,False,False,False
2,내,내,0,True,False,False,False,False
3,채고,가장,1,True,False,False,False,False
4,친한,친한,0,True,False,False,False,False


In [17]:
sentences_region_dummies = pd.get_dummies(sentences_revised, columns=['region'])

sentences_region_dummies.head()

,dialect_form,standard_form,region_강원,region_경상,region_전라,region_제주,region_충청
0,난 니가 내 채고 친한 친구였으문 좋겠단 말이야.,난 네가 내 가장 친한 친구였으면 좋겠단 말이야.,True,False,False,False,False
1,내 마옴을 받아달라는 거 아니야.,내 마음을 받아달라는 거 아니야.,True,False,False,False,False
2,내 마옴을 니가 알아달라는 것 뿐이야.,내 마음을 네가 알아달라는 것 뿐이야.,True,False,False,False,False
3,내가 니 옆에 있다는 것을 기억해 주면 돼.,내가 네 옆에 있다는 것을 기억해 주면 돼.,True,False,False,False,False
4,내가 이딴 고백을 했다구 날 멀리 하진 마.,내가 이런 고백을 했다구 날 멀리 하진 마.,True,False,False,False,False


In [18]:
# 저장
token_region_dummies.to_csv('csv_data/tokens_region_dummies.csv', index=False, encoding='utf-8-sig')
sentences_region_dummies.to_csv('csv_data/sentences_region_dummies.csv', index=False, encoding='utf-8-sig')